In [1]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

4.1.1


In [2]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="Yann_test", pwd="pizza")

In [4]:
conn.query("CREATE OR REPLACE DATABASE dbtest3")

[]

In [5]:
import time
start = time.time()

query_string = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM
'file:///nodes_west_coast.csv'
AS line FIELDTERMINATOR ','
CREATE (n:Node_map {id: toInteger(line.id)})
'''
conn.query(query_string, db='dbtest3')
print(f'elapsed time:  {time.time()-start} s')

elapsed time:  0.9642860889434814 s


In [6]:
import time
start = time.time()
query_string = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM
'file:///edges_west_coast.csv'
AS line FIELDTERMINATOR ','
MATCH (n_orig:Node_map {id: toInteger(line.u)}),(n_dest:Node_map {id: toInteger(line.v)})
CREATE (n_orig)-[:EDGE {id: toInteger(line.id), Co2: toFloat(line.CO2_eq_kg)}]->(n_dest)
'''
conn.query(query_string, db='dbtest3')
print(f'elapsed time:  {time.time()-start} s')

elapsed time:  330.6411199569702 s


In [7]:
import time
start = time.time()
query_string = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM
'file:///edges_west_coast.csv'
AS line FIELDTERMINATOR ','
MATCH (n_orig:Node_map {id: toInteger(line.u)}),(n_dest:Node_map {id: toInteger(line.v)})
CREATE (n_orig)<-[:EDGE {id: toInteger(line.id), Co2: toInteger(line.CO2_eq_kg)}]-(n_dest)
'''
conn.query(query_string, db='dbtest3')
print(f'elapsed time:  {time.time()-start} s')

elapsed time:  348.08279395103455 s


## Shortest path

In [7]:
import time
start = time.time()
query_string = '''
MATCH (n_orig:Node_map {id: 1000035368}), (n_dest:Node_map {id: 1000023449})
CALL gds.alpha.shortestPath.write({
  nodeProjection: 'Node_map',
  relationshipProjection: {
    EDGE: {
      type: 'EDGE',
      properties: 'Co2',
      orientation: 'UNDIRECTED'
    }
  },
  startNode: n_orig,
  endNode: n_dest,
  relationshipWeightProperty: 'Co2',
  writeProperty: 'sssp'
})
YIELD nodeCount, totalCost
RETURN nodeCount,totalCost
'''
result = conn.query(query_string, db='dbtest3')
print(f'elapsed time:  {time.time()-start} s')

elapsed time:  0.27724504470825195 s


In [9]:
result

[<Record nodeCount=243 totalCost=18.4680869803627>]

In [12]:
import pandas as pd
e=pd.read_csv(r'edges_west_coast.csv')

/Users/yann/.conda/envs/Multimodal_freight_USA/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,103,106,113,114,115,116,117,118) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26586 entries, 0 to 26585
Columns: 121 entries, id to v
dtypes: float64(100), int64(3), object(18)
memory usage: 24.5+ MB


In [17]:
e.describe()

,id,dist_miles,duration_h,CO2_eq_kg,"('AR', 'CA')","('AZ', 'IL')","('CA', 'AR')","('CA', 'CO')","('CA', 'FL')","('CA', 'GA')","('CA', 'IL')","('CA', 'MA')","('CA', 'MN')","('CA', 'MO')","('CA', 'NJ')","('CA', 'OH')","('CA', 'TN')","('CA', 'TX')","('CA', 'WA')","('CO', 'CA')","('CO', 'IL')","('CO', 'PA')","('CT', 'TX')","('FL', 'CA')","('FL', 'IL')","('FL', 'MA')","('FL', 'NJ')","('FL', 'TX')","('GA', 'CA')","('GA', 'FL')","('GA', 'IL')","('GA', 'NJ')","('GA', 'PA')","('GA', 'TX')","('GA', 'WA')","('IL', 'AZ')","('IL', 'CA')","('IL', 'CO')","('IL', 'FL')","('IL', 'GA')","('IL', 'MA')","('IL', 'NJ')","('IL', 'OR')","('IL', 'PA')","('IL', 'TX')","('IL', 'UT')","('IL', 'WA')","('MA', 'CA')","('MA', 'FL')","('MA', 'IL')","('MA', 'TX')","('MD', 'IL')","('MN', 'CA')","('MO', 'CA')","('MO', 'OR')","('NC', 'CA')","('NC', 'FL')","('NC', 'IL')","('NJ', 'CA')","('NJ', 'CO')","('NJ', 'FL')","('NJ', 'GA')","('NJ', 'IL')","('NJ', 'MO')","('NJ', 'TX')","('NJ', 'WA')","('NV', 'TX')","('NY', 'FL')","('NY', 'IL')","('OH', 'CA')","('OH', 'TX')","('OR', 'CA')","('OR', 'IL')","('PA', 'IL')","('PA', 'TX')","('SC', 'IL')","('TN', 'CA')","('TX', 'CA')","('TX', 'GA')","('TX', 'IL')","('TX', 'MA')","('TX', 'NC')","('TX', 'NJ')","('TX', 'NV')","('TX', 'OH')","('TX', 'PA')","('TX', 'WA')","('UT', 'CA')","('UT', 'IL')","('WA', 'CA')","('WA', 'GA')","('WA', 'IL')","('WA', 'NJ')","('WA', 'TX')",range1,range2,range3,range4,range5,speed_kmh,DIRECTION,u,v
count,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,26586.000000,34.0,2570.0,2.658600e+04,2.658600e+04
mean,13292.500000,2.988430,0.087254,0.130611,5.478837,4.793083,6.676625,8.844675,5.419451,5.319872,4.632312,5.405230,6.006558,6.248942,5.326771,7.193389,6.205876,5.996678,8.032260,3.628859,3.579294,5.339770,4.617114,3.447929,3.703962,5.223664,4.681965,4.689835,4.135982,8.990033,4.704263,6.765771,6.092638,5.870522,4.949303,5.291876,4.429883,8.036677,6.500283,6.882335,7.646318,8.029338,5.108293,8.549899,6.138546,6.258421,5.294541,3.565865,5.500569,3.707395,4.415833,4.246276,5.450634,5.146617,7.017956,4.088935,7.733305,5.364217,3.610857,5.483446,5.561734,4.910135,4.175553,4.244905,4.467825,4.403173,6.395272,6.067921,4.961718,4.632707,4.755472,4.230974,3.993089,4.532576,4.638130,4.718257,5.072889,4.332110,5.211712,4.960172,6.040141,5.787315,5.628133,5.750325,5.422647,5.861297,5.402450,3.977734,4.485868,4.083110,4.597473,4.092446,4.783508,4.912617,5.949597,5.407803,5.269626,4.609445,4.563107,10.0,2.0,9.031926e+08,9.019222e+08
std,7674.861465,7.680079,0.355207,0.345884,14.003948,12.270503,17.114548,23.038697,13.874632,13.608438,11.839181,13.810789,15.362062,15.966670,13.610794,18.383262,15.878748,15.364299,20.692130,9.320122,9.165804,13.649345,11.828206,8.809842,9.469654,13.462201,12.010051,12.273996,10.578610,